In [ ]:
import numpy as np
import pandas as pd
from fs_s3fs import S3FS

In [ ]:
aws_access_key_id = ''
aws_secret_access_key = ''

filesystem = S3FS(bucket_name='',
                  aws_access_key_id=aws_access_key_id,
                  aws_secret_access_key=aws_secret_access_key)

In [ ]:
data_df = pd.read_parquet(filesystem.openbin('metadata/npz_info_small.pq'))

In [ ]:
eops = data_df.eopatch.unique()

In [ ]:
eops_train = np.random.choice(eops, size=int(len(eops)*0.6), replace=False)
eops_val_test = list(set(eops) - set(eops_train))
eops_test = np.random.choice(eops_val_test, size=int(len(eops_val_test)*0.5), replace=False)
eops_val = list(set(eops) - set(eops_train) - set(eops_test))

In [ ]:
assert set(eops_train).intersection(set(eops_test)) == set()
assert set(eops_train).intersection(set(eops_val)) == set()
assert set(eops_val).intersection(set(eops_test)) == set()
assert set(eops_val).union(set(eops_test)).union(set(eops_train)) == set(eops)

In [ ]:
def set_train_test_val(eopatch, train_set, val_set, test_set):
    if eopatch in train_set:
        return 'train'
    elif eopatch in val_set:
        return 'validation'
    elif eopatch in test_set:
        return 'test'
    raise ValueError(f"Could not find eopatch: {eopatch} in train/test/validation sets.")

In [ ]:
data_df['train_test_validation'] = data_df.eopatch.apply(
    lambda x: set_train_test_val(x, eops_train, eops_val, eops_test))

In [ ]:
with filesystem.openbin('metadata/npz_info_small.pq', 'wb') as f:
    data_df.to_parquet(f)